### Transforming a chess-board to something that a neural network can understand.


The main goal of this notebook is to transfrom a given chess board to something that a computer can comprehend, such as arrays of real numbers. Since the dimensionality of a chess board is 8x8 , it is apparaent that a lot of the features that the neural network is receiving will be in he form of 8x8 arrays which are frequently mentioned as "planes". For example, we can construct many planes, which indicate the position of white and black pieces, E.g. we can have a plane that describes the position of white pawns, another one for black pawns etc. We also include some additional planes in order to create the input dataset ,  which encode information about the pinned pieces of each side, the control of the pieces, castling rights and move turn. Last we also pass a plane which consists of the doubled pawns for the white and black side respectively.



# 4 Vito
However, one needs to be careful to not include too many planes, as this will slow down the training process. In practice, several state-of-the-art neural networks also include the board position, with 14 planes for the positions of pieces, 1 plane for player turn, and 4 planes for castling rights. Additionally those networks also include the previous L board positions (L is integer) , from the last L half-moves.

In [2]:
import nbimporter
import chess
import tensorflow as tf
import numpy as np



### Board --> Pinned pieces


Returns two arrays of respectively white and black pinned pieces coordinates

In [3]:
# Create a chess board

def pinned_pieces(board):
    # Initialize an 8x8 matrix to store information about pinned white pieces
    w_pin_matrix = [[0 for _ in range(8)] for _ in range(8)]

    # Loop through all squares on the board
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            # Check if the piece is pinned for white pieces
            pin = board.is_pinned(chess.WHITE, square)
            if pin:
                # If the piece is pinned, mark the square with the pin
                x, y = chess.square_file(square), chess.square_rank(square)
                w_pin_matrix[x][y] = 1
                
    # Convert the matrix to a numpy array with datatype float32
    w_pin_matrix = np.array(w_pin_matrix, dtype=np.float32)
    
    # Initialize an 8x8 matrix to store information about pinned black pieces
    b_pin_matrix = [[0 for _ in range(8)] for _ in range(8)]

    # Loop through all squares on the board
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            # Check if the piece is pinned for black pieces
            pin = board.is_pinned(chess.BLACK, square)
            if pin:
                # If the piece is pinned, mark the square with the pin
                x, y = chess.square_file(square), chess.square_rank(square)
                b_pin_matrix[x][y] = 1
                
    # Convert the matrix to a numpy array with datatype float32
    b_pin_matrix = np.array(b_pin_matrix, dtype=np.float32)

    # Transpose the matrices and reverse the order of rows to return the result
    return np.transpose(w_pin_matrix)[::-1], np.transpose(b_pin_matrix)[::-1]

# Call the function to get the information about pinned pieces for the current board
# pinned_pieces(board)

### Board --> controlled squares for each side

Returns for each square the amount of pieces having an eye on that square. For example in the starting position none of the white pieces attack h1 , so the value for this square will be 0. However the h2 square is attacked by the rook and therefore its value will be 1. Giving a representation of where the control is on the board to the nerual network gives it an understanding of space utilized in the chessboard, as well as potential targets vs weaknesses.

In [4]:
def control(board):
    # Initialize a numpy array to store the total number of pieces attacking each square on the board
    control_result=np.zeros((8,8))
    
    # Iterate over all squares in the chess board to count total amount of white pieces attacking each square
    for sq in chess.SQUARES:
        # Get the attackers of the current square (sq) for the white pieces
        attacks = board.attackers(chess.WHITE,sq)
        
        # Create an empty numpy array to store the squares being attacked
        attacked_squares_np = np.zeros((8, 8))
        
        for i in range(64):
            # Get the rank and file indices for the current square
            rank_index = i//8
            file_index = i % 8
            square = chess.square(rank_index, file_index)
            
            # Check if the current square is being attacked by any white pieces
            if square in attacks:
                # If yes, mark the corresponding square in the attacked_squares_np array
                attacked_squares_np[chess.square_rank(square), chess.square_file(square)] = 1
        
        # Flip the attacked_squares_np array upside down
        board_of_square_attackers=np.flipud(attacked_squares_np)
        
        # Get the indices of the current square
        id1,id2=sq // 8,sq%8
        
        # Get the total number of attacking pieces on the current square
        pieces_attacking_this_square=(np.sum(attacked_squares_np))
        
        # Update the control_result array with the number of pieces attacking the current square
        control_result[id1,id2]=pieces_attacking_this_square
        
    # Flip the control_result array upside down
    control_result=np.flipud(control_result)
    
    # Initialize a numpy array to store the total number of pieces attacking each square on the board
    control_result2=np.zeros((8,8))
    
    # Iterate over all squares in the chess board to count total amount of black pieces attacking each square
    for sq in chess.SQUARES:
        # Get the attackers of the current square (sq) for the black pieces
        attacks = board.attackers(chess.BLACK,sq)
        
        # Create an empty numpy array to store the squares being attacked
        attacked_squares_np = np.zeros((8, 8))
        
        for i in range(64):
            # Get the rank and file indices for the current square
            rank_index = i//8
            file_index = i % 8
            square = chess.square(rank_index, file_index)
            
            # Check if the current square is being attacked by any black pieces
            if square in attacks:
                # If yes, mark the corresponding square in the attacked_squares_np array
                attacked_squares_np[chess.square_rank(square), chess.square_file(square)] = 1
        # flip the array upside down
        board_of_square_attackers=np.flipud(attacked_squares_np)
        # get the indices of the current square
        id1,id2=sq // 8,sq%8
        # get the total number of attacking pieces on the current square
        pieces_attacking_this_square=(np.sum(attacked_squares_np))
        # update the control_result array with the number of pieces attacking the current square
        control_result2[id1,id2]=pieces_attacking_this_square

    control_result2=np.flipud(control_result2)
    return control_result,control_result2
    

### Board ---> Player turn


This is very very important. When the neural network is receiving a specific board it has to contain some sort of information as well which makes it understand if it is white or black playing. We do this by implementing the "who_plays" part of the metadata which is a plane of zeros if black plays or a plane of ones if white plays. The choice of the (8,8) is inspired by alpha-zero way of implementing turn into the input data while respecting the dimensionality of the plane space

In [5]:
def who_plays(board):
    if board.turn:
        return np.ones((8,8))
    else:
        return np.zeros((8,8))

###  Board ---> Castling rights

It's important to know as well if queenside/kingside castling is available for white/black . If castling is available, the return array is a plane of ones , and ifnot a plane of zeros.

In [6]:


import chess
board = chess.Board()
def castling_rights(board):
    
    white_kingside_castle = board.has_kingside_castling_rights(chess.WHITE)
    wk = np.ones((8,8)) if white_kingside_castle else np.zeros((8,8))
    black_kingside_castle = board.has_kingside_castling_rights(chess.BLACK)
    bk = np.ones((8,8)) if black_kingside_castle else np.zeros((8,8))
    white_queenside_castle = board.has_queenside_castling_rights(chess.WHITE)
    wq = np.ones((8,8)) if white_queenside_castle else np.zeros((8,8))
    black_queenside_castle = board.has_queenside_castling_rights(chess.BLACK)
    bq = np.ones((8,8)) if black_queenside_castle else np.zeros((8,8))
    return wk,bk,wq,bq
    
# castling_rights(board)

### Board --->  Locations of white and black pieces :


The most important piece of the board representation. The location of white and black pieces. In this application the location of pieces is an one hot encoded plane for each of whites/black pieces groups (king,queen(s),rooks,bishops,knights,pawns).  The total amount planes for encoding the pieces location is 12 = (2 white/black) x (6 different groups of pieces).


In [7]:
def board_pieces(board):
    # Initialize the 3D matrix
    board3d = np.zeros((12, 8, 8), dtype="float32")

    # Add the pieces to the matrix
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1.
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1.

    return np.transpose(board3d,(1,2,0))


###  Board --> Passed pawns


 Passed pawns aren't your average pawns. They have a much easier path to promoting to anything they desire. It's important to defend against them or know they exist at least. This function returns two planes which encapsulate white/black passed pawns locations (wow)

In [8]:
squares_index = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7
}

index_squares = {
    0: 'a',
    1: 'b',
    2: 'c',
    3: 'd',
    4: 'e',
    5: 'f',
    6: 'g',
    7: 'h'
}


def find_passed_pawns(board):
    board3d=board_pieces(board)
    white_pawns,black_pawns=board3d[:,:,0],board3d[:,:,6]
    squares_index_mapping = {0: 'a',1: 'b',2: 'c',3: 'd',4: 'e',5: 'f', 6: 'g', 7: 'h'}
    white_passed_pawns = np.zeros((8,8))
    black_passed_pawns = np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            if white_pawns[i][j] == 1:
                if j>0 and j<7:
                    if  not (np.any(black_pawns[:i,j-1]) or np.any(black_pawns[:i,j]) or np.any(black_pawns[:i,j+1])):
                        white_passed_pawns[i][j]=1.
#                         print('white pawn at ',squares_index_mapping[j],8-i,'has neighbouring sides:',black_pawns[:i,j],black_pawns[:i,j],black_pawns[:i,j+1])
                if j==0:
                    if not (np.any(black_pawns[:i,j+1]) or np.any(black_pawns[:i,j])):
                        white_passed_pawns[i][j]=1.
#                         print('white pawn at ',squares_index_mapping[j],8-i,'has neighbouring side:',black_pawns[:i,j-1],black_pawns[:i,j+1])
                if j==7:
                    if not (np.any(black_pawns[:i,j-1]) or np.any(black_pawns[:i,j])):
                        white_passed_pawns[i][j]=1.
#                         print('white pawn at ',squares_index_mapping[j],8-i,'has neighbouring side:',black_pawns[:i,j-1],black_pawns[:i,j])
            elif black_pawns[i][j] == 1:
                if j>0 and j<7:
                    if not (np.any(white_pawns[i:,j-1]) or np.any(white_pawns[i:,j]) or np.any(white_pawns[i:,j+1])):
                        black_passed_pawns[i][j]=+1.
#                         print('the black pawn that exists at ',squares_index_mapping[j],8-i,'has neighbouring sides:',white_pawns[i:,j-1],white_pawns[i:,j],white_pawns[i:,j+1])
                if j==0:
                    if not (np.any(white_pawns[i:,j+1]) or np.any(white_pawns[i:,j])):
                        black_passed_pawns[i][j]=+1.
#                         print('the black pawn at ',squares_index_mapping[j],8-i,'has neighbouring side:',white_pawns[i:,j],white_pawns[i:,j+1])
                if j==7:
                    if not (np.any(white_pawns[i:,j-1]) or np.any(white_pawns[i:,j])):
                        black_passed_pawns[i][j]=+1.
#                         print('the black pawn at ',squares_index_mapping[j],8-i,'has neighbouring side:',white_pawns[i:,j-1],white_pawns[i:,j])
    return white_passed_pawns,black_passed_pawns
# find_passed_pawns(board)

### Board ----> Neural Network input

In [9]:
def square_to_index(square):
    squares_index = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7
    }

    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]


def board_to_network_input(board):
    # 0-5 white pieces
    # 6-11 black pieces
    #12-13 w,b pieces possible moves
    #14-15 w,b passed pawns
    # 16-17 w,b attacked squares
    # 18 half move
    # 19,20 w,b pinned pieces
    # 21,24 castling rights
    # add weighted control
    # Initialize the 3D matrix
    board3d = np.zeros((25, 8, 8), dtype="float32")

    # Add the pieces to the matrix
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1.
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1.

    # Add legal moves ending squares to the matrix
    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[12][i][j] = 1.
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] = 1.
    board.turn = aux
    
    #Add passed pawns
    ch14,ch15=find_passed_pawns(board)
    board3d[14][:][:]=ch14
    board3d[15][:][:]=ch15
    
    #Add controled squares
    ch16,ch17=control(board)
    board3d[16][:][:]=ch16
    board3d[17][:][:]=ch17
    board3d[18][:][:]=who_plays(board)
    
    # Add pinned pieces
    ch19,ch20=pinned_pieces(board)
    board3d[19][:][:]=ch19
    board3d[20][:][:]=ch20
    ch21,ch22,ch23,ch24=castling_rights(board)
    #wk,wq,bk,bq castling rights
    board3d[21][:][:]=ch21
    board3d[22][:][:]=ch22
    board3d[23][:][:]=ch23
    board3d[24][:][:]=ch24
    return np.transpose(board3d,(1,2,0))

    

### The neural network policy output


We have to think at this point that the nerual network policy output has a categorical output for each available move that could arise in a chess board. Neural networks like alpha zero receive some representation of the chess board (X) and predict 2 outputs (Y1,Y2). The first output is the value head, which returns the probability of winning for each side, which is also known as the evaluation bar. Positive values indicate that white is winning and negative values indicate the opposite. The challenging thing to cope with and understand is the second output (Y2), the policy head. This is the output which calculates a probability for every move to be played in the current position. 




After all,depending on the position there are different numbers of possible legal moves. On the other hand we need a constant number of outputs for our network. The general idea to cope with that is to consider every possible starting square and then enumerate all possible moves of a “super-piece” that can move like a queen and a knight and a pawn that promotes. For each of these possible moves we create one output. Naturally there will be outputs that have a positive probability but that correspond to illegal moves. One can cope with that by simply setting these move probabilities to zero and re-compute the remaining probabilities such that their sum equals one.

In [10]:


def all_possible_moves():
    # Define the starting and ending files and rows for chess pieces
    files_start = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    files_end = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    rows_start = ['1', '2', '3', '4', '5', '6', '7', '8']
    rows_end = ['1', '2', '3', '4', '5', '6', '7', '8']
    move_types = ['q', 'r', 'b', 'n']
    
    # Initialize the lists to store all possible moves for each piece
    rook_moves = []
    bishop_moves = []
    pawn_promotion_moves = []
    horse_moves = []
    
    # Loop through all possible combinations of start and end files and rows
    for i1, file_start in enumerate(files_start):
        for i2, file_end in enumerate(files_end):
            for j1, row_start in enumerate(rows_start):
                for j2, row_end in enumerate(rows_end):
                    # Skip the move if start and end positions are the same
                    if not (row_end == row_start and file_end == file_start):
                        # Check if the move is a rook move
                        if row_end == row_start or file_end == file_start:
                            rook_moves.append(file_start + row_start + file_end + row_end)
                        # Check if the move is a bishop move
                        elif np.abs(j2 - j1) == np.abs(i1 - i2):
                            bishop_moves.append(file_start + row_start + file_end + row_end)
                        # Check if the move is a pawn promotion move
                        if np.abs(j2 - j1) == 1 and (np.abs(i2 - i1) <= 1) and ((j1 == 1 and j2 == 0) or (j1 == 6 and j2 == 7)):
                            for move_type in move_types:
                                pawn_promotion_moves.append(file_start + row_start + file_end + row_end + move_type)
                        # Check if the move is a horse move
                        if (np.abs(j2 - j1) == 1 and np.abs(i2 - i1) == 2) or (np.abs(j2 - j1) == 2 and np.abs(i2 - i1) == 1):
                            horse_moves.append(file_start + row_start + file_end + row_end)
    
    # Combine all the possible moves for each piece and sort them
    moves_list = rook_moves + bishop_moves + pawn_promotion_moves + horse_moves
    moves_list = sorted(moves_list)
    
    # Return the combined and sorted list of all possible moves
    return np.array(moves_list)

print('all possible_moves:')
print(all_possible_moves())

all possible_moves:
['a1a2' 'a1a3' 'a1a4' ... 'h8h5' 'h8h6' 'h8h7']
